<a href="https://colab.research.google.com/github/Dadoyen2/Geometry-Optimization/blob/main/geometry_optimization_in_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [322]:
import requests
import math
def get_coordinate(url):
    atoms = []
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()

    # Start parsing for atom data directly after identifying the pattern
    for line in lines:
        parts = line.split()
        # Check if line contains a potential atom data pattern
        if len(parts) >= 4 and parts[3].isalpha():
            try:
                # Attempt to parse the first three values as coordinates
                x, y, z = map(float, parts[:3])
                atom_type = parts[3]
                atoms.append((atom_type, x, y, z))
            except ValueError:
                # Skip lines that don't fit the pattern (ensures generality)
                continue

    return atoms

def print_atom_coordinates(atoms):
    print(f"The input file has: {len(atoms)} atoms")
    print("Atoms and coordinates (in Å):")
    for atom in atoms:
        print(f"{atom[0]:<2} {atom[1]:>8.4f} {atom[2]:>8.4f} {atom[3]:>8.4f}")

# Example URL to use with the general extraction function
url = 'https://raw.githubusercontent.com/Dadoyen2/Geometry-Optimization/main/ethane_dist.mol2'
atoms = get_coordinate(url)
print_atom_coordinates(atoms)




The input file has: 8 atoms
Atoms and coordinates (in Å):
C   -0.7560   0.0500   0.0000
C    0.7960   0.0000   0.0500
H   -1.1804   0.6486   0.7945
H   -1.1004   0.3601  -0.9626
H   -1.2405  -0.9987   0.4781
H    1.1204  -0.3401   0.9526
H    1.1605   1.0187  -0.1581
H    1.1904  -0.6786  -0.7745


In [293]:
import numpy as np
import requests

def bond_length(coord1, coord2):
    return np.linalg.norm(np.array(coord2) - np.array(coord1))

def bond_angle(coord1, coord2, coord3):
    vec1 = np.array(coord1) - np.array(coord2)
    vec2 = np.array(coord3) - np.array(coord2)
    cos_theta = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return np.degrees(np.arccos(cos_theta))



def dihedral_angle(coord1, coord2, coord3, coord4):
    # Define the vectors
    vec1 = np.array(coord1) - np.array(coord2)  # r_AB
    vec2 = np.array(coord3) - np.array(coord2)  # r_BC
    vec3 = np.array(coord4) - np.array(coord3)  # r_CD

    # Calculate normal vectors to planes
    t = np.cross(vec1, vec2)  # Normal to plane ABC
    u = np.cross(vec2, vec3)  # Normal to plane BCD

    # Calculate vector perpendicular to both normal vectors
    v = np.cross(t, u)

    # Compute cos(phi)
    cos_phi = np.dot(t, u) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Compute sin(phi)
    sin_phi = np.dot(vec2 / np.linalg.norm(vec2), v) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Correct numerical inaccuracies (clipping for cos_phi)
    cos_phi = np.clip(cos_phi, -1.0, 1.0)

    # Calculate dihedral angle
    angle_rad = np.arctan2(sin_phi, cos_phi)  # Angle in radians
    angle_degree = np.degrees(angle_rad)     # Convert radians to degrees

    # Normalize angle to the range 0° to 360°
    if angle_degree < 0:
        angle_degree += 360
    angle_rad = np.radians(angle_degree)

  #  return angle_degree, angle_rad

    # Subtract the angle from 180 degrees
    corrected_angle_degree = angle_degree -180
    corrected_angle_rad = np.radians(corrected_angle_degree)

    return corrected_angle_degree, corrected_angle_rad





In [294]:

# Stretching (Bond) Energy Calculation
    # Parameters for bond lengths and constants
bond_params = {
    ('C', 'H'): (350, 1.11),  # Example: k_b and r_0 for C-H bond
    ('C', 'C'): (300, 1.53),  # Example: k_b and r_0 for C-C bond
    #
}

# Define distance cutoffs to distinguish bonded vs non-bonded interactions
bond_cutoffs = {
    ('C', 'H'): (0.5, 1.5),  # Minimum and maximum cutoff for C-H bond detection
    ('C', 'C'): (1.0, 2.0),  # Minimum and maximum cutoff for C-C bond detection
    # Add other cutoffs as needed
}

# Stretching (Bond) Energy Calculation with flexibility for any C-H or hydrocarbon bonds
stretch_energy = 0.0
bond_count = 0
visited_bonds = set()  # To keep track of counted bonds
bonds_new = []
for i in range(len(atoms) - 1):
    for j in range(i + 1, len(atoms)):
        bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))  # Determine bond type dynamically

        # Only consider bond types defined in bond_params and bond_cutoffs
        if bond_type in bond_params and bond_type in bond_cutoffs:
            # Calculate bond length
            r = bond_length(atoms[i][1:], atoms[j][1:])

            # Apply distance cutoff to avoid counting non-bonded pairs
            min_cutoff, max_cutoff = bond_cutoffs[bond_type]
            if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                visited_bonds.add((i, j))  # Mark this bond as visited

                # Get the bond parameters
                k_b, r_0 = bond_params[bond_type]
                bonds_new.append([i,j])
                # Calculate energy for this bond and add to total
                bond_energy = k_b * (r - r_0) ** 2
                stretch_energy += bond_energy
                bond_count += 1

                # Debugging output for verification
                print(f"Bond {bond_type[0]}-{bond_type[1]}: Length = {r:.2f} Å, Energy = {bond_energy:.4f} kcal/mol")

# Output the final results
print(f"\nTotal Stretching (Bond) Energy: {stretch_energy:.4f} kcal/mol")
#print(f"Number of Bonds: {bond_count}")
#print(r)

print(stretch_energy)

Bond C-C: Length = 1.55 Å, Energy = 0.1672 kcal/mol
Bond C-H: Length = 1.08 Å, Energy = 0.2841 kcal/mol
Bond C-H: Length = 1.07 Å, Energy = 0.6071 kcal/mol
Bond C-H: Length = 1.25 Å, Energy = 6.8832 kcal/mol
Bond C-H: Length = 1.02 Å, Energy = 2.9857 kcal/mol
Bond C-H: Length = 1.10 Å, Energy = 0.0237 kcal/mol
Bond C-H: Length = 1.14 Å, Energy = 0.2814 kcal/mol

Total Stretching (Bond) Energy: 11.2323 kcal/mol
11.232337744764989


In [295]:

# Bending energy calculation
bond_distance_threshold = 1.6  # Bonding threshold in angstroms
equilibrium_angle_degrees = 109.5  # Equilibrium bond angle for sp³ carbons

# Define k_a values for bending energy
ka_values_degree = {
    ('H', 'C', 'H'): 35,
    ('H', 'C', 'C'): 35,
    ('C', 'C', 'C'): 60
}

# Initialize bending energy calculations
bend_energy_degrees = 0.0
bend_energy_radians = 0.0
angle_count = 0
angles_new = []  # List to store angles for Wilson B matrix calculation

# Calculate angles and bending energy
for j in range(len(atoms)):
    if atoms[j][0] == 'C':  # Central atom must be carbon
        # Find atoms bonded to the central carbon atom
        bonded_atoms = [
            i for i in range(len(atoms))
            if i != j and bond_length(atoms[i][1:], atoms[j][1:]) <= bond_distance_threshold
        ]

        # Iterate over unique pairs of bonded atoms
        for m in range(len(bonded_atoms)):
            for n in range(m + 1, len(bonded_atoms)):
                i, k = bonded_atoms[m], bonded_atoms[n]

                # Determine the angle type and corresponding k_a value
                angle_type = (atoms[i][0], atoms[j][0], atoms[k][0])
                ka_degree = ka_values_degree.get(angle_type, 35)  # Default to 35 if not found
                angles_new.append([i, j, k])  # Store the angle triplet

                # Calculate bond angle in degrees
                angle_degrees = bond_angle(atoms[i][1:], atoms[j][1:], atoms[k][1:])

                # Calculate bending energy in degrees
                angle_energy_degrees = ka_degree * (angle_degrees - equilibrium_angle_degrees) ** 2
                bend_energy_degrees += angle_energy_degrees

                # Convert energy from degrees to radians
                conversion_factor = (np.pi / 180) ** 2
                angle_energy_radians = angle_energy_degrees * conversion_factor
                bend_energy_radians += angle_energy_radians

                # Increment angle count
                angle_count += 1

                # Print the results
                print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1}: "
                      f"{angle_degrees:>10.3f} {np.radians(angle_degrees):>10.3f} "
                      f"{angle_energy_degrees:>20.6f} {bend_energy_radians:>20.6f}")

# Print total bending energy
#print(f"\nTotal bending energy in degrees: {bend_energy_degrees:.6f} kcal/mol")
print(f"Total bending energy in radians: {bend_energy_radians:.6f} kcal/mol")
#print(f"Total number of angles calculated: {angle_count}")

# Angles list for Wilson B matrix
#print("\nAngles list for Wilson B matrix:")
#print(angles_new)
print(bend_energy_radians)


C2 - C1 - H3:    112.717      1.967           362.187201             0.110329
C2 - C1 - H4:    111.123      1.939            92.193260             0.138412
C2 - C1 - H5:    110.354      1.926            25.538209             0.146192
H3 - C1 - H4:    112.009      1.955           220.283660             0.213294
H3 - C1 - H5:     91.792      1.602         10975.255770             3.556548
H4 - C1 - H5:    117.587      2.052          2289.193178             4.253876
C1 - C2 - H6:    110.962      1.937            74.801757             4.276662
C1 - C2 - H7:    107.137      1.870           195.496814             4.336214
C1 - C2 - H8:    109.998      1.920             8.667359             4.338854
H6 - C2 - H7:    111.782      1.951           182.190328             4.394352
H6 - C2 - H8:    109.435      1.910             0.146832             4.394397
H7 - C2 - H8:    107.443      1.875           148.139085             4.439523
Total bending energy in radians: 4.439523 kcal/mol
4.43952274368

In [296]:
import numpy as np

def bond_length(coord1, coord2):
    """Calculate bond length between two atoms."""
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

def calculate_stretching_gradient(atoms, bond_params, bond_cutoffs):
    """
    Calculate bond stretching energy and its gradients.

    Parameters:
    - atoms: List of atoms with their coordinates
    - bond_params: Dictionary with bond parameters (k_b, r_0)
    - bond_cutoffs: Dictionary with bond cutoff distances

    Returns:
    - stretch_energy: Total bond stretching energy
    - stretching_gradients: Gradients of bond stretching energy for all atoms
    """
    coords = [np.array(atom[1:]) for atom in atoms]  # Ensure coordinates are numpy arrays
    n_atoms = len(atoms)
    visited_bonds = set()
    stretching_gradients = np.zeros((3, n_atoms))
    stretch_energy = 0.0
    bond_data = []

    for i in range(n_atoms - 1):
        for j in range(i + 1, n_atoms):
            bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))

            if bond_type in bond_params and bond_type in bond_cutoffs:
                r = bond_length(coords[i], coords[j])
                min_cutoff, max_cutoff = bond_cutoffs[bond_type]

                if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                    visited_bonds.add((i, j))

                    k_b, r_0 = bond_params[bond_type]
                    delta_r = r - r_0

                    # Stretching energy
                    bond_energy = k_b * delta_r ** 2
                    stretch_energy += bond_energy

                    # Gradient calculation using the formula g_stretch(r) = 2 * k_b * delta_r * d(r)/dX
                    grad_factor = 2 * k_b * delta_r / r
                    grad_i = grad_factor * (coords[i] - coords[j])
                    grad_j = -grad_i

                    # Accumulate gradients
                    stretching_gradients[:, i] += grad_i
                    stretching_gradients[:, j] += grad_j

                    # Store bond information for printing
                    bond_data.append((atoms[i][0], i + 1, atoms[j][0], j + 1, r, bond_energy))

    # Print results
    #print("\nList of all bond lengths with energies:")
   # for atom1, idx1, atom2, idx2, length, energy in bond_data:
       # print(f"{atom1}{idx1} - {atom2}{idx2}: {length:>10.3f} {energy:>10.6f}")

    #print(f"\nTotal Bond Stretching Energy: {stretch_energy:.6f} kcal/mol")
    #print("Number of bonds calculated:", len(bond_data))

    print("\nAnalytical gradient of stretching energy:")
    for atom_index, (atom, grad) in enumerate(zip(atoms, stretching_gradients.T)):
        print(f"{atom[0]}{atom_index + 1:>2} {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return stretching_gradients

#stretching_gradients = calculate_stretching_gradient(atoms, bond_params, bond_cutoffs)


In [297]:
import numpy as np

def bond_length(coord1, coord2):
    """Calculate bond length between two atoms."""
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

def bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold):
    """
    Calculate bending energy and its gradients using the cross product method.

    Parameters:
    - atoms: List of atoms with coordinates
    - ka_values_degree: Dictionary with force constants for angle types
    - bond_distance_threshold: Threshold to determine bonded atoms
    """
    equilibrium_angle_radians = np.radians(109.5)  # Default equilibrium angle for sp3 hybridization
    bending_gradients = np.zeros((len(atoms), 3))  # Gradient array
    total_bending_energy = 0.0

    for j in range(len(atoms)):  # Loop through central atoms
        if atoms[j][0] == 'C':  # Focus on carbon atoms as central atoms
            bonded_atoms = [
                i for i in range(len(atoms))
                if i != j and bond_length(atoms[i][1:], atoms[j][1:]) <= bond_distance_threshold
            ]

            for m in range(len(bonded_atoms)):
                for n in range(m + 1, len(bonded_atoms)):
                    i, k = bonded_atoms[m], bonded_atoms[n]
                    angle_type = (atoms[i][0], atoms[j][0], atoms[k][0])
                    ka_degree = ka_values_degree.get(angle_type, 35)

                    # Compute vectors
                    r_BA = np.array(atoms[j][1:]) - np.array(atoms[i][1:])
                    r_BC = np.array(atoms[j][1:]) - np.array(atoms[k][1:])
                    p = np.cross(r_BA, r_BC)  # Cross product of the vectors

                    r_AB = np.linalg.norm(r_BA)
                    r_BC_norm = np.linalg.norm(r_BC)
                    norm_p = np.linalg.norm(p)

                    cos_theta = np.dot(r_BA, r_BC) / (r_AB * r_BC_norm)
                    theta = np.arccos(np.clip(cos_theta, -1.0, 1.0))
                    delta_theta = theta - equilibrium_angle_radians

                    # Bending energy
                    energy = ka_degree * delta_theta**2
                    total_bending_energy += energy

                    # Gradient calculation
                    grad_A = -np.cross(r_BA, p) / (r_AB**2 * norm_p) * 2 * ka_degree * delta_theta
                    grad_C = np.cross(r_BC, p) / (r_BC_norm**2 * norm_p) * 2 * ka_degree * delta_theta
                    grad_B = -(grad_A + grad_C)

                    # Accumulate gradients
                    bending_gradients[i] += grad_A
                    bending_gradients[j] += grad_B
                    bending_gradients[k] += grad_C

    # Print results
   # print(f"Total Bending Energy: {total_bending_energy:.6f} kcal/mol")
    print("Analytical Gradient of Bending Energy:")
    for atom_index, grad in enumerate(bending_gradients):
        print(f"{atoms[atom_index][0]}{atom_index + 1:>2} {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return bending_gradients



bond_distance_threshold = 1.6

#bending_gradients = bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold)


In [298]:
import numpy as np

# Mock torsion energy parameters
torsion_params = {
    "v_n": 0.3,  # kcal/mol, torsional barrier
    "n": 3,      # Periodicity of torsion angle
}

# Helper functions
def bond_length(coord1, coord2):
    """Calculate the bond length between two atoms."""
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

def dihedral_angle(a, b, c, d):
    """Calculate the dihedral angle in degrees and radians."""
    b1 = np.array(b) - np.array(a)
    b2 = np.array(c) - np.array(b)
    b3 = np.array(d) - np.array(c)

    n1 = np.cross(b1, b2)
    n2 = np.cross(b2, b3)

    n1 /= np.linalg.norm(n1)
    n2 /= np.linalg.norm(n2)

    m1 = np.cross(n1, b2 / np.linalg.norm(b2))

    x = np.dot(n1, n2)
    y = np.dot(m1, n2)

    angle_rad = np.arctan2(y, x)
    angle_deg = np.degrees(angle_rad)

    return angle_deg, angle_rad

# Main function
def calculate_and_print_torsions(atoms):
    coords = [atom[1:] for atom in atoms]  # Extract coordinates
    n_atoms = len(atoms)
    torsions = []
    connectivity = [[] for _ in range(n_atoms)]  # Bond connectivity matrix

    # Build connectivity matrix (bond list)
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < 1.6:  # Threshold for bonding
                connectivity[i].append(j)
                connectivity[j].append(i)

    # Find all torsions by iterating over atom pairs
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue
                    # Calculate dihedral angle
                    angle_deg, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])

                    # Calculate torsion energy in degrees and radians
                    torsion_energy_deg = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * np.radians(angle_deg)))
                    torsion_energy_rad = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * angle_rad))

                    # Append torsion data (atom indices, angles, energies)
                    torsions.append((i, j, k, l, angle_deg, angle_rad, torsion_energy_deg, torsion_energy_rad))

    # Print torsions and total energies
    torsion_energy_total_deg = 0.0
    torsion_energy_total_rad = 0.0
    print("\nList of all torsion angles with energies:")
    for (i, j, k, l, angle_deg, angle_rad, torsion_energy_deg, torsion_energy_rad) in torsions:
        torsion_energy_total_deg += torsion_energy_deg
        torsion_energy_total_rad += torsion_energy_rad
        print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1} - {atoms[l][0]}: "
              f"{angle_deg:>10.3f} {angle_rad: >10.3f} {torsion_energy_deg:>20.6f}")

    # Print total torsion energy
    print(f"\nTotal torsion energy (calculated using degrees): {torsion_energy_total_deg:.6f} kcal/mol")
    print(f"Total torsion energy (calculated using radians): {torsion_energy_total_rad:.6f} kcal/mol")
    print(f"Number of torsions calculated: {len(torsions)}")

    # Return total torsion energy
    return torsion_energy_total_rad





# Call the function to calculate torsion energy
torsion_energy = calculate_and_print_torsions(atoms)

# Convert to Python float (if needed)
total_torsion_energy = float(torsion_energy)


print(f"Torsion Energy (Radians): {total_torsion_energy:.6f} kcal/mol")
print(type(total_torsion_energy))





List of all torsion angles with energies:
H3 - C1 - C2 - H:    -56.162     -0.980             0.006038
H3 - C1 - C2 - H:     66.111      1.154             0.015228
H3 - C1 - C2 - H:   -177.400     -3.096             0.002775
H4 - C1 - C2 - H:    177.164      3.092             0.003302
H4 - C1 - C2 - H:    -60.563     -1.057             0.000130
H4 - C1 - C2 - H:     55.925      0.976             0.006802
H5 - C1 - C2 - H:     44.877      0.783             0.089239
H5 - C1 - C2 - H:    167.150      2.917             0.065381
H5 - C1 - C2 - H:    -76.362     -1.333             0.103519

Total torsion energy (calculated using degrees): 0.292415 kcal/mol
Total torsion energy (calculated using radians): 0.292415 kcal/mol
Number of torsions calculated: 9
Torsion Energy (Radians): 0.292415 kcal/mol
<class 'float'>


In [299]:

 """

# Main function
def calculate_total_torsion_energy(atoms):

    coords = [atom[1:] for atom in atoms]  # Extract coordinates
    n_atoms = len(atoms)
    connectivity = [[] for _ in range(n_atoms)]  # Bond connectivity matrix

    # Build connectivity matrix (bond list)
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < 1.6:  # Threshold for bonding
                connectivity[i].append(j)
                connectivity[j].append(i)

    # Initialize total torsion energy
    torsion_energy_total_rad = 0.0

    # Find all torsions by iterating over atom pairs
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue
                    # Calculate dihedral angle
                    _, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])

                    # Calculate torsion energy in radians
                    torsion_energy_rad = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * angle_rad))

                    # Add to total torsion energy
                    torsion_energy_total_rad += torsion_energy_rad

    # Return total torsion energy
    return torsion_energy_total_rad




# Call the function to calculate torsion energy
torsion_energy_total = calculate_total_torsion_energy(atoms)

# Print the total torsion energy
print(f"\nTorsion Energy (Radians): {torsion_energy_total:.6f} kcal/mol")
print(type(torsion_energy_total))
"""


'\n\n# Main function\ndef calculate_total_torsion_energy(atoms):\n \n   coords = [atom[1:] for atom in atoms]  # Extract coordinates\n   n_atoms = len(atoms)\n   connectivity = [[] for _ in range(n_atoms)]  # Bond connectivity matrix\n\n   # Build connectivity matrix (bond list)\n   for i in range(n_atoms):\n       for j in range(i + 1, n_atoms):\n           if bond_length(coords[i], coords[j]) < 1.6:  # Threshold for bonding\n               connectivity[i].append(j)\n               connectivity[j].append(i)\n\n   # Initialize total torsion energy\n   torsion_energy_total_rad = 0.0\n\n   # Find all torsions by iterating over atom pairs\n   for j in range(n_atoms):\n       for a in range(len(connectivity[j])):\n           k = connectivity[j][a]\n           if k < j:\n               continue\n           for b in range(len(connectivity[j])):\n               i = connectivity[j][b]\n               if i == k:\n                   continue\n               for c in range(len(connectivity[k])):\

In [300]:
import numpy as np

def bond_length(coord1, coord2):
    """Calculate bond length between two atoms."""
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

def dihedral_angle(p1, p2, p3, p4):
    """
    Calculate the dihedral angle between four points in space.
    Returns the angle in degrees and radians.
    """
    b1 = np.array(p2) - np.array(p1)
    b2 = np.array(p3) - np.array(p2)
    b3 = np.array(p4) - np.array(p3)

    # Normalize b2 for better stability
    b2_norm = b2 / np.linalg.norm(b2)

    v1 = np.cross(b1, b2_norm)
    v2 = np.cross(b2_norm, b3)
    v1 /= np.linalg.norm(v1)
    v2 /= np.linalg.norm(v2)

    angle_rad = np.arccos(np.clip(np.dot(v1, v2), -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)

    # Check sign of the dihedral
    if np.dot(np.cross(v1, v2), b2_norm) < 0.0:
        angle_rad = -angle_rad
        angle_deg = -angle_deg

    return angle_deg, angle_rad

def calculate_and_print_torsions_with_gradient(atoms, torsion_params):
    """
    Calculate torsion angles, torsion energies, and their gradients.

    Parameters:
    - atoms: List of atoms with their coordinates
    - torsion_params: Dictionary with torsion parameters (v_n, n)

    Returns:
    - torsion_energy_total_rad: Total torsion energy in radians
    - dihedrals: List of dihedrals (atom indices)
    - torsion_gradients: Gradients of torsion energy for all atoms
    """
    coords = [np.array(atom[1:]) for atom in atoms]  # Ensure coordinates are numpy arrays
    n_atoms = len(atoms)
    torsions = []
    dihedrals = []  # Store dihedrals for Wilson B matrix
    connectivity = [[] for _ in range(n_atoms)]  # Bond connectivity matrix
    torsion_gradients = np.zeros((3, n_atoms))  # Gradient array

    # Build connectivity matrix (bond list)
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < 1.6:  # Threshold for bonding
                connectivity[i].append(j)
                connectivity[j].append(i)

    # Find all torsions and calculate energies
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue

                    # Calculate dihedral angle
                    angle_deg, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])

                    # Save dihedral for Wilson B matrix
                    dihedrals.append((i, j, k, l))

                    # Calculate torsion energy
                    torsion_energy_rad = torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * angle_rad))
                    torsions.append((i, j, k, l, angle_deg, angle_rad, torsion_energy_rad))

                    # Gradient calculation
                    grad_term = -torsion_params['n'] * torsion_params['v_n'] * np.sin(torsion_params['n'] * angle_rad)
                    b1 = coords[j] - coords[i]
                    b2 = coords[k] - coords[j]
                    b3 = coords[l] - coords[k]

                    t = np.cross(b1, b2)
                    u = np.cross(b2, b3)
                    norm_t = np.linalg.norm(t)
                    norm_u = np.linalg.norm(u)

                    term1 = np.cross(t, b2) / (norm_t**2 * np.linalg.norm(b2))
                    term2 = np.cross(-u, b2) / (norm_u**2 * np.linalg.norm(b2))

                    # Gradients for each atom
                    grad_i = grad_term * np.cross(term1, b2)
                    grad_j = grad_term * (np.cross(coords[k] - coords[i], term1) + np.cross(term2, b3))
                    grad_k = grad_term * (np.cross(term1, b1) + np.cross(coords[l] - coords[j], term2))
                    grad_l = grad_term * np.cross(term2, b2)

                    # Accumulate gradients
                    torsion_gradients[:, i] += grad_i
                    torsion_gradients[:, j] += grad_j
                    torsion_gradients[:, k] += grad_k
                    torsion_gradients[:, l] += grad_l

    # Print torsions and gradients
    torsion_energy_total_rad = 0.0
    #print("\nList of all torsion angles with energies:")
    #for (i, j, k, l, angle_deg, angle_rad, torsion_energy_rad) in torsions:
     #   torsion_energy_total_rad += torsion_energy_rad
      #  print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1} - {atoms[l][0]}: "
       #       f"{angle_deg:>10.3f} {angle_rad:>10.3f} {torsion_energy_rad:>20.6f}")

    #print(f"\nTotal torsion energy (calculated using radians): {torsion_energy_total_rad:.6f} kcal/mol")
    #print(f"Number of torsions calculated: {len(torsions)}")

    print("\nAnalytical gradient of torsional energy:")
    for atom_index, (atom, grad) in enumerate(zip(atoms, torsion_gradients.T)):
        print(f"{atom[0]}{atom_index + 1:>2} {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return torsion_gradients
torsion_gradients = calculate_and_print_torsions_with_gradient(atoms, torsion_params)




Analytical gradient of torsional energy:
C 1   0.078501   0.168224  -2.268434
C 2  -0.032176  -0.055102   0.943633
H 3  -0.026348  -0.474459   0.343397
H 4  -0.007406  -0.335219  -0.105341
H 5  -0.025146   0.675363   1.455908
H 6  -0.011907  -0.586197  -0.216600
H 7  -0.007162   0.060289   0.282587
H 8   0.031645   0.547101  -0.435151


In [301]:
#internal coordinate
"""
internal_coords = bond_count + angle_count +  len(torsion_n)
cartesian_coords = 3 * len(atoms)

print(f"Internal Coordinates: {internal_coords}")
print(f"Cartesian Coordinates: {cartesian_coords}")
"""

'\ninternal_coords = bond_count + angle_count +  len(torsion_n)\ncartesian_coords = 3 * len(atoms)\n\nprint(f"Internal Coordinates: {internal_coords}")\nprint(f"Cartesian Coordinates: {cartesian_coords}")\n'

In [302]:
Total_energy_vanderwaals = []

# Function to calculate van der Waals energy and print as it iterates
def calculate_and_print_vdw_energy(atoms, bonds_new, angles_new):
    # Define epsilon and sigma values for Hydrogen and Carbon
    epsilon_values = {'H': 0.03, 'C': 0.07}  # in kcal/mol
    sigma_values = {'H': 1.20, 'C': 1.75}    # in Angstroms

    total_energy = 0.0

    print(f"{'Atom Pair':<15} {'Distance (Å)':>15} {'Energy (kcal/mol)':>20}")

    N = len(atoms)

    # Iterate over all unique atom pairs
    for i in range(N):
        for j in range(i + 1, N):
            calc_vdw = True  # Assume we will calculate vdW for this pair
            # Exclude bonded pairs
            for bond in bonds_new:
                if i in bond and j in bond:
                    calc_vdw = False
                    break
            # Exclude pairs involved in angles
            if calc_vdw:
                for angle in angles_new:
                    if i in angle and j in angle:
                        calc_vdw = False
                        break

            # Retrieve atom types and coordinates
            atom1_type, x1, y1, z1 = atoms[i]
            atom2_type, x2, y2, z2 = atoms[j]
            r_ij = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

            if calc_vdw:
                # Retrieve epsilon and sigma values
                epsilon_i = epsilon_values.get(atom1_type)
                epsilon_j = epsilon_values.get(atom2_type)
                sigma_i = sigma_values.get(atom1_type)
                sigma_j = sigma_values.get(atom2_type)

                if epsilon_i is None or epsilon_j is None:
                    raise ValueError(f"Unknown atom type: {atom1_type} or {atom2_type}")

                # Compute mixed epsilon and sigma using geometric mean
                epsilon_ij = math.sqrt(epsilon_i * epsilon_j)
                sigma_ij = 2*math.sqrt(sigma_i * sigma_j)

                # Compute Lennard-Jones potential
                term12 = (sigma_ij / r_ij) ** 12
                term6 = (sigma_ij / r_ij) ** 6
                energy_lj = 4 * epsilon_ij * (term12 - term6)

                # Accumulate the total energy
                total_energy += energy_lj
                energy = energy_lj
            else:
                # For uncalculated pairs, energy is zero
                energy = 0.0

            # Print the current pair details
            print(f"{atom1_type} {i + 1:<3}- {atom2_type} {j + 1:<3}: {r_ij:>15.4f} {energy:>20.4f}")
            # Append the energy to the list
    Total_energy_vanderwaals.append(total_energy)

    print(f"\nTotal van der Waals Energy: {total_energy:.4f} kcal/mol")
    return total_energy
# Example Input
# Calculate and print van der Waals energy
van_der_waal_ener= calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)
#print(bonds_new)

print(van_der_waal_ener)
print(torsion_energy)
print(bend_energy_radians)
print(stretch_energy)


Atom Pair          Distance (Å)    Energy (kcal/mol)
C 1  - C 2  :          1.5536               0.0000
C 1  - H 3  :          1.0815               0.0000
C 1  - H 4  :          1.0684               0.0000
C 1  - H 5  :          1.2502               0.0000
C 1  - H 6  :          2.1402               0.0000
C 1  - H 7  :          2.1532               0.0000
C 1  - H 8  :          2.2179               0.0000
C 2  - H 3  :          2.2093               0.0000
C 2  - H 4  :          2.1798               0.0000
C 2  - H 5  :          2.3082               0.0000
C 2  - H 6  :          1.0176               0.0000
C 2  - H 7  :          1.1018               0.0000
C 2  - H 8  :          1.1384               0.0000
H 3  - H 4  :          1.7824               0.0000
H 3  - H 5  :          1.6785               0.0000
H 3  - H 6  :          2.5092              -0.0215
H 3  - H 7  :          2.5543              -0.0258
H 3  - H 8  :          3.1375              -0.0192
H 4  - H 5  :          1.9853

In [303]:
import math
import numpy as np

# Total van der Waals energy container
Total_energy_vanderwaals = []

def calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new):
    # Define epsilon and sigma values for Hydrogen and Carbon
    epsilon_values = {'H': 0.03, 'C': 0.07}  # in kcal/mol
    sigma_values = {'H': 1.20, 'C': 1.75}    # in Angstroms

    total_energy = 0.0
    gradients = np.zeros((len(atoms), 3))  # Initialize gradients
    van_der_waal_gradients = []
    #print(f"{'Atom Pair':<15} {'Distance (Å)':>15} {'Energy (kcal/mol)':>20}")

    N = len(atoms)

    # Iterate over all unique atom pairs
    for i in range(N):
        for j in range(i + 1, N):
            calc_vdw = True  # Assume we will calculate vdW for this pair
            # Exclude bonded pairs
            for bond in bonds_new:
                if i in bond and j in bond:
                    calc_vdw = False
                    break
            # Exclude pairs involved in angles
            if calc_vdw:
                for angle in angles_new:
                    if i in angle and j in angle:
                        calc_vdw = False
                        break

            # Retrieve atom types and coordinates
            atom1_type, x1, y1, z1 = atoms[i]
            atom2_type, x2, y2, z2 = atoms[j]
            r_ij_vector = np.array([x1 - x2, y1 - y2, z1 - z2])
            r_ij = np.linalg.norm(r_ij_vector)

            if calc_vdw:
                # Retrieve epsilon and sigma values
                epsilon_i = epsilon_values.get(atom1_type)
                epsilon_j = epsilon_values.get(atom2_type)
                sigma_i = sigma_values.get(atom1_type)
                sigma_j = sigma_values.get(atom2_type)

                if epsilon_i is None or epsilon_j is None:
                    raise ValueError(f"Unknown atom type: {atom1_type} or {atom2_type}")

                # Compute mixed epsilon and sigma using geometric mean
                epsilon_ij = math.sqrt(epsilon_i * epsilon_j)
                sigma_ij = 2 * math.sqrt(sigma_i * sigma_j)

                # Compute Lennard-Jones potential energy
                term12 = (sigma_ij / r_ij) ** 12
                term6 = (sigma_ij / r_ij) ** 6
                energy_lj = 4 * epsilon_ij * (term12 - term6)
                total_energy += energy_lj

                # Compute Lennard-Jones gradient
                A_ij = 4 * epsilon_ij * sigma_ij**12
                B_ij = 4 * epsilon_ij * sigma_ij**6
                grad_prefactor = (-12 * A_ij / r_ij**14 + 6 * B_ij / r_ij**8)
                gradient = grad_prefactor * r_ij_vector

                # Accumulate gradients
                gradients[i] += gradient
                gradients[j] -= gradient  # Opposite direction for atom j

                # Print the current pair details
             #   print(f"{atom1_type} {i + 1:<3}- {atom2_type} {j + 1:<3}: {r_ij:>15.4f} {energy_lj:>20.4f}")
            #else:
            #    energy_lj = 0.0

   # Total_energy_vanderwaals.append(total_energy)
    van_der_waal_gradients.append(gradients)
    # Print total energy and gradients
    #print(f"\nTotal van der Waals Energy: {total_energy:.4f} kcal/mol")
    print("\nAnalytical Gradients (kcal/mol/Å):")
    for idx, (atom, grad) in enumerate(zip(atoms, gradients)):
        print(f"{atom[0]}{idx + 1:<2} : {grad[0]:>10.6f} {grad[1]:>10.6f} {grad[2]:>10.6f}")

    return van_der_waal_gradients



# Calculate and print van der Waals energy and gradients
#van_der_waal_gradients = calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new)


In [304]:
def calculate_total_energy(van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy):
    """
    Calculate the total energy of the initial structure by summing up
    the individual energy components.

    Parameters:
        van_der_waal_ener (float): Van der Waals energy (kcal/mol).
        torsion_energy (float): Torsion energy (kcal/mol).
        bend_energy_radians (float): Bending energy in radians (kcal/mol).
        stretch_energy (float): Stretching energy (kcal/mol).

    Returns:
        float: Total energy of the initial structure (kcal/mol).
    """
    # Calculate the total energy
    total_energy = (
        van_der_waal_ener +
        torsion_energy +
        bend_energy_radians +
        stretch_energy
    )

    # Print individual energy components
    print(f"Van der Waals Energy: {van_der_waal_ener:.4f} kcal/mol")
    print(f"Torsion Energy: {torsion_energy:.4f} kcal/mol")
    print(f"Bending Energy (Radians): {bend_energy_radians:.4f} kcal/mol")
    print(f"Stretching Energy: {stretch_energy:.4f} kcal/mol")

    # Print the total energy
    print(f"\nTotal Energy of Initial Structure: {total_energy:.4f} kcal/mol")

    return total_energy



# Call the function and calculate the total energy
total_energy_initial = calculate_total_energy(
    van_der_waal_ener,
    torsion_energy,
    bend_energy_radians,
    stretch_energy
)


Van der Waals Energy: -0.1982 kcal/mol
Torsion Energy: 0.2924 kcal/mol
Bending Energy (Radians): 4.4395 kcal/mol
Stretching Energy: 11.2323 kcal/mol

Total Energy of Initial Structure: 15.7661 kcal/mol


In [305]:

def ensure_numpy_array(array):
    """Convert input to a numpy array if it isn't already."""
    if not isinstance(array, np.ndarray):
        array = np.array(array)
    return array

def reshape_to_match(array, target_shape):
    """
    Reshape or transpose an array to match the target shape.
    Handles cases like 1-dimensional expansion or axis swaps.
    """
    if array.shape == target_shape:
        return array
    if len(array.shape) == 3 and array.shape[0] == 1:
        array = array[0]  # Remove the first dimension
    if array.T.shape == target_shape:
        return array.T
    try:
        array = array.reshape(target_shape)
    except ValueError:
        raise ValueError(f"Cannot reshape or transpose array of shape {array.shape} to match {target_shape}")
    return array



stretching_gradients = calculate_stretching_gradient(atoms, bond_params, bond_cutoffs)
bending_gradient = bending_energy_with_gradients(atoms, ka_values_degree, bond_distance_threshold)
torsion_energy = calculate_and_print_torsions_with_gradient(atoms, torsion_params)
van_der_waal_gradients = np.array(calculate_and_print_vdw_energy_with_gradient(atoms, bonds_new, angles_new))



# Print original shapes for debugging
#print("Stretching Gradient Shape:", stretching_gradients.shape)
#print("Bending Gradient Shape:", bending_gradient.shape)
#print("Torsion Energy Shape:", torsion_energy.shape)
#print("Van der Waals Gradient Shape:", van_der_waal_gradients.shape)

# Determine the target shape dynamically (assume torsion_energy as base)
target_shape = torsion_energy.shape

# Reshape all components to match the target shape
try:
    stretching_gradients_reshaped = reshape_to_match(stretching_gradients, target_shape)
    bending_gradient_reshaped = reshape_to_match(bending_gradient, target_shape)
    van_der_waal_gradients_reshaped = reshape_to_match(van_der_waal_gradients, target_shape)
except ValueError as e:
    print("Error in reshaping arrays:", e)
    raise

# Ensure all shapes are compatible for addition
if (stretching_gradients_reshaped.shape == bending_gradient_reshaped.shape ==
        torsion_energy.shape == van_der_waal_gradients_reshaped.shape):
    # Calculate total energy by summing all components
    Total_energy = (stretching_gradients_reshaped +
                    bending_gradient_reshaped +
                    torsion_energy +
                    van_der_waal_gradients_reshaped)

    # Transpose the result for better readability
    Total_energy_transposed = Total_energy.T.tolist()

    # Store energy data for later use
    energy_data = []
    for idx, (atom, energy_row) in enumerate(zip(atoms, Total_energy_transposed)):
        energy_data.append({
            'atom': atom[0],
            'index': idx + 1,
            'energy': energy_row
        })
else:
    print("Shapes are incompatible for addition after reshaping.")
    energy_data = []

# Access or print energy data later
print("\nTotal Analytical gradient Energy for Initial Structure (in kcal/mol/Å):")
for entry in energy_data:
    print(f"{entry['atom']}{entry['index']:<2}: {entry['energy'][0]:>10.6f} {entry['energy'][1]:>10.6f} {entry['energy'][2]:>10.6f}")


Analytical gradient of stretching energy:
C 1   6.666498 102.297338 -49.614523
C 2  29.788419  -4.910144  71.088132
H 3   7.825438 -11.037481 -14.649647
H 4   9.398544  -8.462510  26.268984
H 5 -38.041793 -82.341442  37.539280
H 6 -20.609681  21.607128 -57.343704
H 7  -1.903969  -5.321189   1.087012
H 8   6.876544 -11.831701 -14.375534
Analytical Gradient of Bending Energy:
C 1   7.891121   6.942520 -27.775624
C 2   0.574346  -3.291197   6.095162
H 3  -9.905003 -18.677791   8.781426
H 4   5.433210   5.156700  -0.282677
H 5  -3.298284  10.292252  19.233352
H 6   0.375758  -2.255585  -0.984955
H 7  -2.609603   0.048441  -4.333749
H 8   1.538455   1.784661  -0.732934

Analytical gradient of torsional energy:
C 1   0.078501   0.168224  -2.268434
C 2  -0.032176  -0.055102   0.943633
H 3  -0.026348  -0.474459   0.343397
H 4  -0.007406  -0.335219  -0.105341
H 5  -0.025146   0.675363   1.455908
H 6  -0.011907  -0.586197  -0.216600
H 7  -0.007162   0.060289   0.282587
H 8   0.031645   0.547101

In [306]:


# Example molecular structure: List of atoms

# Calculate the total number of degrees of freedom
num_atoms = len(atoms)
n = 3 * num_atoms  # Each atom contributes 3 degrees of freedom

# Initialize the inverse Hessian matrix as a scaled identity matrix
M_k = np.eye(n) / 300  # Scale by 300 for demonstration purposes

# Print the initial guess for the inverse Hessian in the desired format
print("Initial Guess for the Inverse Hessian (M_k):")
for i, row in enumerate(M_k, start=1):
    print(f"Row {i:<3}: " + " ".join(f"{val:>10.6f}" for val in row))



gradient_flat = np.array([entry['energy'] for entry in energy_data]).flatten()

# Calculate the predicted structure change
p_k = -np.dot(M_k, gradient_flat)

# Reshape gradients and p_k for better readability
gradient_reshaped = gradient_flat.reshape(len(atoms), 3)
p_k_reshaped = p_k.reshape(len(atoms), 3)

# Print the predicted structure change
print("\n***** Geometry optimization cycle number   1 *****\n")
print("Predicted structure change p_k = -M_k grad V(r_k):")
for idx, (atom, p_row, grad_row) in enumerate(zip(atoms, p_k_reshaped, gradient_reshaped)):
    print(f"{atom[0]:<2} {p_row[0]:>10.6f} {p_row[1]:>10.6f} {p_row[2]:>10.6f} ")





Initial Guess for the Inverse Hessian (M_k):
Row 1  :   0.003333   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000
Row 2  :   0.000000   0.003333   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000
Row 3  :   0.000000   0.000000   0.003333   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000
Row 4  :   0.000000   0.000000   0.000000   0.003333   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   0.00

In [307]:
import numpy as np

# Function to calculate total energy
def calculate_total_energy(van_der_waal_ener, total_torsion_energy, bend_energy_radians, stretch_energy):
    """
    Calculate the total energy of the initial structure by summing up
    the individual energy components.

    Parameters:
        van_der_waal_ener (float or ndarray): Van der Waals energy (kcal/mol).
        total_torsion_energy (float or ndarray): Torsion energy (kcal/mol).
        bend_energy_radians (float or ndarray): Bending energy in radians (kcal/mol).
        stretch_energy (float or ndarray): Stretching energy (kcal/mol).

    Returns:
        float: Total energy of the initial structure (kcal/mol).
    """
    # Ensure inputs are scalars (convert NumPy arrays to floats if needed)
    van_der_waal_ener = float(van_der_waal_ener) if isinstance(van_der_waal_ener, np.ndarray) else van_der_waal_ener
    total_torsion_energy = float(total_torsion_energy) if isinstance(total_torsion_energy, np.ndarray) else total_torsion_energy
    bend_energy_radians = float(bend_energy_radians) if isinstance(bend_energy_radians, np.ndarray) else bend_energy_radians
    stretch_energy = float(stretch_energy) if isinstance(stretch_energy, np.ndarray) else stretch_energy

    # Calculate total energy
    total_energy = van_der_waal_ener + total_torsion_energy + bend_energy_radians + stretch_energy

    # Print individual energy components
    print(f"Van der Waals Energy: {van_der_waal_ener:.4f} kcal/mol")
    print(f"Torsion Energy: {total_torsion_energy:.4f} kcal/mol")
    print(f"Bending Energy (Radians): {bend_energy_radians:.4f} kcal/mol")
    print(f"Stretching Energy: {stretch_energy:.4f} kcal/mol")
    print(f"\nTotal Energy of Initial Structure: {total_energy:.4f} kcal/mol")

    return total_energy




# Calculate total energy
total_energy = calculate_total_energy(van_der_waal_ener, total_torsion_energy, bend_energy_radians, stretch_energy)

print(f"\nTotal Energy: {total_energy:.4f} kcal/mol")
print(type(total_energy))


Van der Waals Energy: -0.1982 kcal/mol
Torsion Energy: 0.2924 kcal/mol
Bending Energy (Radians): 4.4395 kcal/mol
Stretching Energy: 11.2323 kcal/mol

Total Energy of Initial Structure: 15.7661 kcal/mol

Total Energy: 15.7661 kcal/mol
<class 'numpy.float64'>


In [320]:
import numpy as np

# Flatten atomic coordinates into r_k (geometry vector)
r_k = np.array([coord for atom in atoms for coord in atom[1:]])

# Flatten the gradient data
gradient_flat = np.array([entry['energy'] for entry in energy_data]).flatten()

# Initialize the inverse Hessian matrix (scaled identity matrix)
n = len(r_k)  # Total degrees of freedom
M_k = np.eye(n) / 300  # Scaled identity matrix

# Compute the search direction p_k
p_k = -np.dot(M_k, gradient_flat)

# Reshape gradients and p_k for readability
gradient_reshaped = gradient_flat.reshape(len(atoms), 3)
p_k_reshaped = p_k.reshape(len(atoms), 3)

# Print predicted structure change
print("\n***** Geometry optimization cycle number   1 *****\n")
print("Predicted structure change p_k = -M_k grad V(r_k):")
for idx, (atom, p_row) in enumerate(zip(atoms, p_k_reshaped)):
    print(f"{atom[0]:<2} {p_row[0]:>10.6f} {p_row[1]:>10.6f} {p_row[2]:>10.6f}")

# Define the objective function
def objective_function(alpha, r_k, p_k):
    """
    Compute the total energy of the updated structure during line search.

    Parameters:
        alpha (float): Step size along the search direction.
        r_k (np.array): Current geometry vector (flattened).
        p_k (np.array): Search direction vector (flattened).

    Returns:
        float: Total energy of the updated structure.
    """
    # Update geometry based on alpha and search direction
    updated_geometry = r_k + alpha * p_k

    # Simulated energies (mock values)
    van_der_waal = np.sum(updated_geometry) * van_der_waal_ener
    total_torsion = np.sum(updated_geometry) * total_torsion_energy
    bend_energy = np.sum(updated_geometry) * bend_energy_radians
    stretch_energ = np.sum(updated_geometry) * stretch_energy

    # Calculate total energy based on updated geometry
    total_energy = van_der_waal + total_torsion + bend_energy + stretch_energ
    return total_energy

# Perform line search
def line_search(r_k, p_k, alpha_init=0.8, beta=0.8, tolerance=1e-5):
    """
    Perform line search to find the optimal step size.

    Parameters:
        r_k (np.array): Current geometry vector (flattened).
        p_k (np.array): Search direction vector (flattened).
        alpha_init (float): Initial step size.
        beta (float): Reduction factor for step size.
        tolerance (float): Convergence tolerance.

    Returns:
        tuple: Optimal alpha and corresponding energy.
    """
    alpha = alpha_init
    energy_prev = objective_function(0, r_k, p_k)
    while True:
        energy_new = objective_function(alpha, r_k, p_k)
        print(f"Line search: alpha and energy: {alpha:>10.6f} {energy_new:>10.6f}")
        if abs(energy_new - energy_prev) < tolerance or alpha < tolerance:
            return alpha, energy_new
        energy_prev = energy_new
        alpha *= beta  # Reduce alpha

# Perform the line search to find optimal alpha
alpha, energy = line_search(r_k, p_k)

# Update geometry
r_k1 = r_k + alpha * p_k
print(f"\nOptimal alpha: {alpha:.6f}")
print(f"Energy at optimal alpha: {energy:.6f}")



***** Geometry optimization cycle number   1 *****

Predicted structure change p_k = -M_k grad V(r_k):
C   -0.048787  -0.364694   0.265529
C   -0.101102   0.027521  -0.260423
H    0.006509   0.100712   0.018436
H   -0.050081   0.012129  -0.086385
H    0.137575   0.237857  -0.194217
H    0.068185  -0.062566   0.195196
H    0.015657   0.017481   0.009960
H   -0.027956   0.031559   0.051905
Line search: alpha and energy:   0.800000   6.779430

Optimal alpha: 0.800000
Energy at optimal alpha: 6.779430


In [309]:
import numpy as np

# Flatten atomic coordinates into r_k (geometry vector)
r_k = np.array([coord for atom in atoms for coord in atom[1:]])

# Flatten the gradient data
gradient_flat = np.array([entry['energy'] for entry in energy_data]).flatten()

# Initialize the inverse Hessian matrix (scaled identity matrix)
n = len(r_k)  # Total degrees of freedom
M_k = np.eye(n) / 300  # Scaled identity matrix

# Compute the search direction p_k
p_k = -np.dot(M_k, gradient_flat)

# Reshape gradients and p_k for readability
gradient_reshaped = gradient_flat.reshape(len(atoms), 3)
p_k_reshaped = p_k.reshape(len(atoms), 3)

# Print predicted structure change
print("\n***** Geometry optimization cycle number   1 *****\n")
print("Predicted structure change p_k = -M_k grad V(r_k):")
for idx, (atom, p_row) in enumerate(zip(atoms, p_k_reshaped)):
    print(f"{atom[0]:<2} {p_row[0]:>10.6f} {p_row[1]:>10.6f} {p_row[2]:>10.6f}")

# Define the objective function (energy computation)
def objective_function(alpha, r_k, p_k):
    updated_geometry = r_k + alpha * p_k
    # Mock energy function: Sum of squares (replace with actual energy function)
    return np.sum(updated_geometry)

# Perform line search
def line_search(r_k, p_k, alpha_init=0.8, beta=0.8, tolerance=1e-5):
    alpha = alpha_init
    energy_prev = objective_function(0, r_k, p_k)
    while True:
        energy_new = objective_function(alpha, r_k, p_k)
        print(f"Line search: alpha and energy: {alpha:>10.6f} {energy_new:>10.6f}")
        if energy_new < energy_prev or alpha < tolerance:
            return alpha, energy_new
        alpha *= beta  # Reduce alpha

# Perform the line search to find optimal alpha
alpha, energy = line_search(r_k, p_k)

# Update geometry
r_k1 = r_k + alpha * p_k

# Reshape the updated geometry for readability
r_k1_reshaped = r_k1.reshape(len(atoms), 3)

# Print the updated structure
print("\nNew structure r_k+1 = r_k + s_k = r_k + alpha_k p_k:")
for idx, (atom, r_row) in enumerate(zip(atoms, r_k1_reshaped)):
    print(f"{atom[0]:<2} {r_row[0]:>10.6f} {r_row[1]:>10.6f} {r_row[2]:>10.6f}")



***** Geometry optimization cycle number   1 *****

Predicted structure change p_k = -M_k grad V(r_k):
C   -0.048787  -0.364694   0.265529
C   -0.101102   0.027521  -0.260423
H    0.006509   0.100712   0.018436
H   -0.050081   0.012129  -0.086385
H    0.137575   0.237857  -0.194217
H    0.068185  -0.062566   0.195196
H    0.015657   0.017481   0.009960
H   -0.027956   0.031559   0.051905
Line search: alpha and energy:   0.800000   0.430000
Line search: alpha and energy:   0.640000   0.430000

New structure r_k+1 = r_k + s_k = r_k + alpha_k p_k:
C   -0.787224  -0.183404   0.169938
C    0.731295   0.017614  -0.116671
H   -1.176234   0.713055   0.806299
H   -1.132452   0.367863  -1.017886
H   -1.152452  -0.846472   0.353801
H    1.164038  -0.380142   1.077525
H    1.170521   1.029888  -0.151726
H    1.172508  -0.658402  -0.741281


In [310]:


# Flatten atomic coordinates into r_k (geometry vector)
r_k = np.array([coord for atom in atoms for coord in atom[1:]])

# Flatten the gradient data
gradient_flat = np.array([entry['energy'] for entry in energy_data]).flatten()

# Initialize the inverse Hessian matrix (scaled identity matrix)
n = len(r_k)  # Total degrees of freedom
M_k = np.eye(n) / 300  # Scaled identity matrix

# Compute the search direction p_k
p_k = -np.dot(M_k, gradient_flat)

# Reshape gradients and p_k for readability
gradient_reshaped = gradient_flat.reshape(len(atoms), 3)
p_k_reshaped = p_k.reshape(len(atoms), 3)

# Print predicted structure change
print("\n***** Geometry optimization cycle number   1 *****\n")
print("Predicted structure change p_k = -M_k grad V(r_k):")
for idx, (atom, p_row) in enumerate(zip(atoms, p_k_reshaped)):
    print(f"{atom[0]:<2} {p_row[0]:>10.6f} {p_row[1]:>10.6f} {p_row[2]:>10.6f}")

# Define the objective function (energy computation)
def objective_function(alpha, r_k, p_k):
    updated_geometry = r_k + alpha * p_k
    # Mock energy function: Sum of squares (replace with actual energy function)
    return np.sum(updated_geometry)

# Perform line search with fixed alpha
def perform_line_search(r_k, p_k, fixed_alpha):
    """
    Perform line search with a fixed alpha value.

    Parameters:
        r_k (np.array): Current geometry vector.
        p_k (np.array): Search direction vector.
        fixed_alpha (float): Pre-determined step size.

    Returns:
        float: Energy at the updated geometry.
    """
    energy = objective_function(fixed_alpha, r_k, p_k)
    print(f"Using fixed alpha = {fixed_alpha:.6f}, Energy = {energy:.6f}")
    return fixed_alpha, energy

# Use alpha = 0.262144 for the line search
optimal_alpha = 0.262144
alpha, energy = perform_line_search(r_k, p_k, optimal_alpha)

# Update geometry
r_k1 = r_k + alpha * p_k

# Reshape the updated geometry for readability
r_k1_reshaped = r_k1.reshape(len(atoms), 3)

# Print the updated structure
print("\nNew structure r_k+1 = r_k + s_k = r_k + alpha_k p_k:")
for idx, (atom, r_row) in enumerate(zip(atoms, r_k1_reshaped)):
    print(f"{atom[0]:<2} {r_row[0]:>10.6f} {r_row[1]:>10.6f} {r_row[2]:>10.6f}")



***** Geometry optimization cycle number   1 *****

Predicted structure change p_k = -M_k grad V(r_k):
C   -0.048787  -0.364694   0.265529
C   -0.101102   0.027521  -0.260423
H    0.006509   0.100712   0.018436
H   -0.050081   0.012129  -0.086385
H    0.137575   0.237857  -0.194217
H    0.068185  -0.062566   0.195196
H    0.015657   0.017481   0.009960
H   -0.027956   0.031559   0.051905
Using fixed alpha = 0.262144, Energy = 0.430000

New structure r_k+1 = r_k + s_k = r_k + alpha_k p_k:
C   -0.768789  -0.045602   0.069607
C    0.769497   0.007215  -0.018268
H   -1.178694   0.675001   0.799333
H   -1.113529   0.363280  -0.985245
H   -1.204436  -0.936347   0.427187
H    1.138274  -0.356501   1.003769
H    1.164604   1.023283  -0.155489
H    1.183072  -0.670327  -0.760893


In [311]:
# Mock energy calculation function
def calculate_total_energy(van_der_waal_ener, total_torsion_energy, bend_energy_radians, stretch_energy):
    """
    Simulate energy calculation by summing up the individual components.

    Parameters:
        van_der_waal_ener (float): Van der Waals energy.
        total_torsion_energy (float): Torsion energy.
        bend_energy_radians (float): Bending energy.
        stretch_energy (float): Stretching energy.

    Returns:
        float: Simulated total energy.
    """
    # Total energy as a sum of all components
    total_energy = van_der_waal_ener + total_torsion_energy + bend_energy_radians + stretch_energy
    return total_energy


# Define the objective function
def objective_function(alpha, r_k, p_k):
    """
    Compute the total energy of the updated structure during line search.

    Parameters:
        alpha (float): Step size along the search direction.
        r_k (np.array): Current geometry vector (flattened).
        p_k (np.array): Search direction vector (flattened).

    Returns:
        float: Total energy of the updated structure.
    """
    # Update geometry based on alpha and search direction
    updated_geometry = r_k + alpha * p_k

    # Simulated energies (mock values, replace with real calculations if needed)
    van_der_waal_ener = np.sum(updated_geometry**2) * 0.1
    total_torsion_energy = np.sum(updated_geometry**2) * 0.2
    bend_energy_radians = np.sum(updated_geometry**2) * 0.3
    stretch_energy = np.sum(updated_geometry**2) * 0.4

    # Calculate total energy based on updated geometry
    total_energy = calculate_total_energy(van_der_waal_ener, total_torsion_energy, bend_energy_radians, stretch_energy)
    return total_energy


# Perform line search
def line_search(r_k, p_k, alpha_init=0.8, beta=0.8, tolerance=1e-5):
    """
    Perform line search to find the optimal step size.

    Parameters:
        r_k (np.array): Current geometry vector (flattened).
        p_k (np.array): Search direction vector (flattened).
        alpha_init (float): Initial step size.
        beta (float): Reduction factor for step size.
        tolerance (float): Convergence tolerance.

    Returns:
        tuple: Optimal alpha and corresponding energy.
    """
    alpha = alpha_init
    energy_prev = objective_function(0, r_k, p_k)
    while True:
        energy_new = objective_function(alpha, r_k, p_k)
        print(f"Line search: alpha and energy: {alpha:>10.6f} {energy_new:>10.6f}")
        if abs(energy_new - energy_prev) < tolerance or alpha < tolerance:
            return alpha, energy_new
        energy_prev = energy_new
        alpha *= beta  # Reduce alpha


# Mock atomic data for testing

r_k = np.array([coord for atom in atoms for coord in atom[1:]])  # Flatten atomic coordinates

# Mock gradient data
gradient_flat = np.random.random(len(r_k))  # Random mock gradient data
n = len(r_k)  # Total degrees of freedom
M_k = np.eye(n) / 300  # Scaled identity matrix

# Compute the search direction p_k
p_k = -np.dot(M_k, gradient_flat)

# Perform the line search to find optimal alpha
alpha, energy = line_search(r_k, p_k)

# Update geometry
r_k1 = r_k + alpha * p_k
print(f"\nOptimal alpha: {alpha:.6f}")
print(f"Energy at optimal alpha: {energy:.6f}")


Line search: alpha and energy:   0.800000  15.850921
Line search: alpha and energy:   0.640000  15.851268
Line search: alpha and energy:   0.512000  15.851548
Line search: alpha and energy:   0.409600  15.851774
Line search: alpha and energy:   0.327680  15.851957
Line search: alpha and energy:   0.262144  15.852103
Line search: alpha and energy:   0.209715  15.852221
Line search: alpha and energy:   0.167772  15.852316
Line search: alpha and energy:   0.134218  15.852392
Line search: alpha and energy:   0.107374  15.852453
Line search: alpha and energy:   0.085899  15.852502
Line search: alpha and energy:   0.068719  15.852541
Line search: alpha and energy:   0.054976  15.852572
Line search: alpha and energy:   0.043980  15.852597
Line search: alpha and energy:   0.035184  15.852617
Line search: alpha and energy:   0.028147  15.852633
Line search: alpha and energy:   0.022518  15.852646
Line search: alpha and energy:   0.018014  15.852656
Line search: alpha and energy:   0.014412  15.

In [312]:
import numpy as np

# Define a mock function to simulate energy calculation based on geometry
def calculate_energy(updated_geometry):
    """
    Simulate energy calculation based on the updated geometry.
    For demonstration, this uses a simple quadratic function.

    Parameters:
        updated_geometry (np.array): Updated geometry vector.

    Returns:
        float: Simulated total energy.
    """
    # Example: Sum of squares of updated_geometry
    return np.sum(updated_geometry**2)

# Define the objective function
def objective_function(alpha, r_k, p_k):
    """
    Compute the total energy of the updated structure during line search.

    Parameters:
        alpha (float): Step size along the search direction.
        r_k (np.array): Current geometry vector (flattened).
        p_k (np.array): Search direction vector (flattened).

    Returns:
        float: Total energy of the updated structure.
    """
    # Update geometry based on alpha and search direction
    updated_geometry = r_k + alpha * p_k
    # Calculate energy based on updated geometry
    total_energy = calculate_energy(updated_geometry)
    return total_energy

# Perform line search
def line_search(r_k, p_k, alpha_init=0.8, beta=0.8, tolerance=1e-5):
    """
    Perform line search to find the optimal step size.

    Parameters:
        r_k (np.array): Current geometry vector (flattened).
        p_k (np.array): Search direction vector (flattened).
        alpha_init (float): Initial step size.
        beta (float): Reduction factor for step size.
        tolerance (float): Convergence tolerance.

    Returns:
        tuple: Optimal alpha and corresponding energy.
    """
    alpha = alpha_init
    energy_prev = objective_function(0, r_k, p_k)
    while True:
        energy_new = objective_function(alpha, r_k, p_k)
        print(f"Line search: alpha and energy: {alpha:>10.6f} {energy_new:>10.6f}")
        if energy_new < energy_prev or alpha < tolerance:
            return alpha, energy_new
        energy_prev = energy_new
        alpha *= beta  # Reduce alpha



# Perform the line search to find optimal alpha
alpha, energy = line_search(r_k, p_k)

# Update geometry
r_k1 = r_k + alpha * p_k
print(f"\nOptimal alpha: {alpha:.6f}")
print(f"Energy at optimal alpha: {energy:.6f}")


Line search: alpha and energy:   0.800000  15.850921

Optimal alpha: 0.800000
Energy at optimal alpha: 15.850921


In [313]:
import numpy as np

# Function to calculate total energy using individual components
def calculate_total_energy(van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy):
    """
    Calculate the total energy of the initial structure by summing up
    the individual energy components.
    """
    # Ensure all inputs are scalar values
    if not all(isinstance(val, (int, float, np.float64)) for val in [van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy]):
        raise TypeError("All inputs to calculate_total_energy must be scalar values.")

    total_energy = (
        van_der_waal_ener +
        torsion_energy +
        bend_energy_radians +
        stretch_energy
    )
    return total_energy

# Define the objective function using total energy calculation
def objective_function(alpha, r_k, p_k, van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy):
    """
    Compute the total energy of the updated geometry as the objective function.
    """
    # Updated geometry after applying alpha scaling
    updated_geometry = r_k + alpha * p_k

    # Recalculate energy components for the new geometry if needed.
    # (This would involve re-calculating van_der_waal_ener, torsion_energy, etc.)
    # For simplicity, this example assumes fixed energy values.

    # Use the total energy function to compute the energy
    total_energy = calculate_total_energy(
        van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy
    )
    return float(total_energy)  # Ensure it returns a scalar

# Perform line search
def line_search(r_k, p_k, van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy, alpha_init=0.8, beta=0.8, tolerance=1e-2):
    """
    Perform a backtracking line search to find the optimal step size alpha.
    """
    alpha = alpha_init
    energy_prev = objective_function(0, r_k, p_k, van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy)
    while True:
        energy_new = objective_function(alpha, r_k, p_k, van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy)
        print(f"Line search: alpha and energy: {alpha:>10.6f} {energy_new:>10.6f}")
        if energy_new < energy_prev or alpha < tolerance:
            return alpha, energy_new
        alpha *= beta  # Reduce alpha

# Flatten atomic coordinates into r_k (geometry vector)
r_k = np.array([coord for atom in atoms for coord in atom[1:]])

# Flatten the gradient data
gradient_flat = np.array([entry['energy'] for entry in energy_data]).flatten()

# Initialize the inverse Hessian matrix (scaled identity matrix)
n = len(r_k)  # Total degrees of freedom
M_k = np.eye(n) / 300  # Scaled identity matrix

# Compute the search direction p_k
p_k = -np.dot(M_k, gradient_flat)

# Reshape gradients and p_k for readability
gradient_reshaped = gradient_flat.reshape(len(atoms), 3)
p_k_reshaped = p_k.reshape(len(atoms), 3)

# Print predicted structure change
print("\n***** Geometry optimization cycle number   1 *****\n")
print("Predicted structure change p_k = -M_k grad V(r_k):")
for idx, (atom, p_row) in enumerate(zip(atoms, p_k_reshaped)):
    print(f"{atom[0]:<2} {p_row[0]:>10.6f} {p_row[1]:>10.6f} {p_row[2]:>10.6f}")

# Perform the line search to find optimal alpha
alpha, energy = line_search(r_k, p_k, van_der_waal_ener, torsion_energy, bend_energy_radians, stretch_energy)

# Update geometry
r_k1 = r_k + alpha * p_k

# Reshape the updated geometry for readability
r_k1_reshaped = r_k1.reshape(len(atoms), 3)

# Print the updated structure
print("\nNew structure r_k+1 = r_k + s_k = r_k + alpha_k p_k:")
for idx, (atom, r_row) in enumerate(zip(atoms, r_k1_reshaped)):
    print(f"{atom[0]:<2} {r_row[0]:>10.6f} {r_row[1]:>10.6f} {r_row[2]:>10.6f}")



***** Geometry optimization cycle number   1 *****

Predicted structure change p_k = -M_k grad V(r_k):
C   -0.048787  -0.364694   0.265529
C   -0.101102   0.027521  -0.260423
H    0.006509   0.100712   0.018436
H   -0.050081   0.012129  -0.086385
H    0.137575   0.237857  -0.194217
H    0.068185  -0.062566   0.195196
H    0.015657   0.017481   0.009960
H   -0.027956   0.031559   0.051905


TypeError: All inputs to calculate_total_energy must be scalar values.

In [ ]:
#print_atom_coordinates(atoms)
#print(f"\nTotal Stretching (Bond) Energy: {stretch_energy:.4f} kcal/mol")
#print(f"Number of Bonds: {bond_count}")
#print(f"\nTotal bending energy in degrees: {bend_energy_degrees:.6f} kcal/mol")
#print(f"Total bending energy in radians: {bend_energy_radians:.6f} kcal/mol")
#print(f"Total number of angles calculated: {angle_count}")
#print(f"Internal Coordinates: {internal_coords}")
#print(f"Cartesian Coordinates: {cartesian_coords}")
#torsion_energy, dihedrals = calculate_and_print_torsions(atoms)
#calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)


In [ ]:
def bond_angle_and_derivatives(A, B, C):
    """
    Calculate the bond angle theta_ABC and its partial derivatives.

    Parameters:
    A, B, C: numpy arrays representing the 3D coordinates of atoms A, B, and C.

    Returns:
    theta (float): Bond angle in degrees.
    derivatives (tuple): Partial derivatives with respect to x, y, z for A, B, and C.
    """
    # Vectors
    r_BA = A - B
    r_BC = C - B

    # Magnitudes
    norm_BA = np.linalg.norm(r_BA)
    norm_BC = np.linalg.norm(r_BC)

    # Cross product and its magnitude
    p = np.cross(r_BA, r_BC)
    norm_p = np.linalg.norm(p)

    # Partial derivatives
    partial_A = (np.cross(r_BA, p) / (norm_BA**2 * norm_p))
    partial_C = (-np.cross(r_BC, p) / (norm_BC**2 * norm_p))
    partial_B = (-partial_A - partial_C)  # Central atom contribution


    return partial_A, partial_B, partial_C

def Wilson_Matrix_dihedral(rA, rB, rC, rD):
    # Vectors connecting the atoms
    r_AB = rA - rB
    r_BC = rB - rC
    r_CD = rC - rD
    r_AC = rA - rC
    r_BD = rB - rD

    # Cross products
    t = np.cross(r_AB, r_BC)
    u = np.cross(r_BC, r_CD)

    # Magnitudes squared of t and u
    t_squared = np.dot(t, t)
    u_squared = np.dot(u, u)

    # Compute gradients
    wil_matr_A = np.cross(t, r_BC) / ( np.linalg.norm(t)**2 * np.linalg.norm(r_BC))
    wil_matr_D = np.cross(-u, r_BC) / (np.linalg.norm(u)**2 * np.linalg.norm(r_BC))
    wil_matr_B = np.cross(r_AC, wil_matr_A)+  np.cross(wil_matr_D, r_CD,)
    wil_matr_C = np.cross(wil_matr_A, r_AB) +  np.cross(r_BD, wil_matr_D)

    return wil_matr_A, wil_matr_B, wil_matr_C, wil_matr_D

def compute_wilson_b_matrix_bonds(atoms, bonds):
    """Compute the Wilson B matrix for bonds (only bonds)."""
    num_atoms = len(atoms)
    n_q = len(bonds)  # Number of bonds
    n_x = 3 * num_atoms  # Number of Cartesian coordinates (3 per atom)

    B_matrix = np.zeros((n_q, n_x))

    for i, (atom_A_idx, atom_B_idx) in enumerate(bonds):
        atom_A = np.array(atoms[atom_A_idx][1:])  # Convert to numpy array
        atom_B = np.array(atoms[atom_B_idx][1:])  # Convert to numpy array

        # Compute bond length
        r_AB = np.linalg.norm(atom_A - atom_B)

        # Compute derivatives with respect to Cartesian coordinates
        d_r_AB_d_A = (atom_A - atom_B) / r_AB
        d_r_AB_d_B = -d_r_AB_d_A

        # Fill the B-matrix for bond i
        B_matrix[i, 3*atom_A_idx:3*(atom_A_idx+1)] = d_r_AB_d_A
        B_matrix[i, 3*atom_B_idx:3*(atom_B_idx+1)] = d_r_AB_d_B

    return B_matrix

def compute_wilson_b_matrix_angles(atoms, angles):
    """Compute the Wilson B matrix for angles (only angles)."""
    num_atoms = len(atoms)
    n_q = len(angles)  # Number of angles
    n_x = 3 * num_atoms  # Number of Cartesian coordinates (3 per atom)

    B_matrix = np.zeros((n_q, n_x))

    for i, (atom_A_idx, atom_B_idx, atom_C_idx) in enumerate(angles):
        atom_A = np.array(atoms[atom_A_idx][1:])  # Convert to numpy array
        atom_B = np.array(atoms[atom_B_idx][1:])  # Convert to numpy array
        atom_C = np.array(atoms[atom_C_idx][1:])  # Convert to numpy array

        # Compute angle derivatives using bond_angle_and_derivatives
        partial_A, partial_B, partial_C = bond_angle_and_derivatives(
            atom_A, atom_B, atom_C)

        # Fill the B-matrix for angle i
        B_matrix[i, 3*atom_A_idx:3*(atom_A_idx+1)] = partial_A
        B_matrix[i, 3*atom_B_idx:3*(atom_B_idx+1)] = partial_B
        B_matrix[i, 3*atom_C_idx:3*(atom_C_idx+1)] = partial_C

    return B_matrix


def compute_wilson_b_matrix_dihedrals(atoms, dihedrals):


    num_atoms = len(atoms)
    n_q = len(dihedrals)  # Number of dihedrals
    n_x = 3 * num_atoms  # Number of Cartesian coordinates

    B_matrix = np.zeros((n_q, n_x))

    for i, (atom_A_idx, atom_B_idx, atom_C_idx, atom_D_idx) in enumerate(dihedrals):
        atom_A = np.array(atoms[atom_A_idx][1:])
        atom_B = np.array(atoms[atom_B_idx][1:])
        atom_C = np.array(atoms[atom_C_idx][1:])
        atom_D = np.array(atoms[atom_D_idx][1:])

        # Compute derivatives
        partial_A, partial_B, partial_C, partial_D = Wilson_Matrix_dihedral(
            atom_A, atom_B, atom_C, atom_D
        )

        # Fill the B-matrix
        B_matrix[i, 3 * atom_A_idx : 3 * (atom_A_idx + 1)] = partial_A
        B_matrix[i, 3 * atom_B_idx : 3 * (atom_B_idx + 1)] = partial_B
        B_matrix[i, 3 * atom_C_idx : 3 * (atom_C_idx + 1)] = partial_C
        B_matrix[i, 3 * atom_D_idx : 3 * (atom_D_idx + 1)] = partial_D

    return B_matrix

def compute_wilson_b_matrix(atoms, bonds, angles, dihedrals):



    # Compute the bond part of the B matrix
    B_matrix_bonds = compute_wilson_b_matrix_bonds(atoms, bonds)

    # Compute the angle part of the B matrix
    B_matrix_angles = compute_wilson_b_matrix_angles(atoms, angles)

    # Compute the dihedral part of the B matrix
    B_matrix_dihedrals = compute_wilson_b_matrix_dihedrals(atoms, dihedrals)

    # Combine bond, angle, and dihedral contributions
    B_matrix = np.vstack((B_matrix_bonds, B_matrix_angles, B_matrix_dihedrals))

    return B_matrix


# G matrix calculation (product of B and its transpose)
def calculate_g_matrix(B_matrix):
    G_matrix = np.dot(B_matrix, B_matrix.T)
    return G_matrix

def calculate_inverse_matrix(G_matrix):
  inverse_matrix = np.linalg.inv(G_matrix)
  return inverse_matrix
# Print matrix with row numbering
def print_matrix_with_numbering(matrix, label):
    print(f"\n{label}:")
    for i, row in enumerate(matrix, start=1):
        row_str = ' '.join(f"{val:>10.5f}" for val in row)
        print(f"{i:>2}: {row_str}")



# Example atoms list (replace this with actual atom data)


# Calculate Wilson B matrix for the methane structure
B_matrix = compute_wilson_b_matrix(atoms, bonds_new, angles_new, dihedrals)

# Calculate the G matrix
G_matrix = calculate_g_matrix(B_matrix)

inverse_matrix=  calculate_inverse_matrix(G_matrix)

# Print the Wilson B matrix and G matrix with row numbering
print_matrix_with_numbering(B_matrix, "Wilson B Matrix at the initial structure")
print_matrix_with_numbering(matrix=G_matrix, label="G matrix, the product of B with its own transpose (square, dimension 28) ")
#print_matrix_with_numbering(inverse_matrix, "Inverse G Matrix (square matrix with dimension 28) at the initial structure")





# New Section